Detection of local minima (> 1m) on a rescaled DEM (2m)
Last update: 13/03/2025

In [15]:
# Load modules
import numpy as np
from osgeo import gdal
from numpy.lib.stride_tricks import as_strided
import time
import csv

In [16]:
# Define functions
def load_data(file_path):
    """
    Input: DEM (raster) file path 
    Output: DEM (array) and geotransform information
    """
    try:
        gdal.UseExceptions()
        data = gdal.Open(file_path)
        if data is None:
            raise FileNotFoundError(f"Could not open file: {file_path}")
        data_array = data.ReadAsArray()
        geotransform = data.GetGeoTransform()
        return data_array, geotransform
    except Exception as e:
        raise RuntimeError(f"Error loading DEM: {e}")

def normalize_data(data):
    """
    Input: DEM (array)
    Output: Normalized DEM (array)
    """
    return (data - np.min(data)) / (np.max(data) - np.min(data)) # 0-1 range

def detect_local_minima(dem_normalized, window_size):
    """
    Input: Normalized DEM (array) and focal window size
    Output: Local minima (mask)
    """
    rows, cols = dem_normalized.shape
    shape = (rows - window_size + 1, cols - window_size + 1, window_size, window_size)
    strides = (dem_normalized.strides[0], dem_normalized.strides[1],
               dem_normalized.strides[0], dem_normalized.strides[1])
    sliding_windows = as_strided(dem_normalized, shape, strides)

    local_minima = np.min(sliding_windows, axis=(2, 3))
    local_maxima = np.max(sliding_windows, axis=(2, 3))
    center_values = dem_normalized[window_size // 2:rows - window_size // 2,
                                    window_size // 2:cols - window_size // 2]
    dem_min, dem_max = np.min(dem_normalized), np.max(dem_normalized)
    mask = (local_minima == center_values) & (local_maxima - local_minima <= 1 / (dem_max - dem_min))
    return mask

def extract_coordinates(mask, geotransform, window_size):
    """
    Input: Local minima (mask), geotransform information and focal window size
    Output: Local minima (coordinates)
    """
    coordinates = []
    for i, j in zip(*np.nonzero(mask)):
        center_x = window_size // 2 + j
        center_y = window_size // 2 + i
        geo_x = geotransform[0] + center_x * geotransform[1] + center_y * geotransform[2]
        geo_y = geotransform[3] + center_x * geotransform[4] + center_y * geotransform[5]
        coordinates.append((len(coordinates) + 1, geo_x, geo_y))
    return coordinates

def save_to_csv(data, filename, headers):
    """
    Input: Local minima (coordinates), filename and headers
    Output: Local minima (CSV)
    """
    try:
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(headers)
            writer.writerows(data)
        print(f"Results saved to {filename}")
    except Exception as e:
        raise RuntimeError(f"Error writing to CSV: {e}")

In [18]:
# File paths
dem_path = r"C:\Users\sarah\OneDrive\Documenten\GitHub\thesis-cds-github\Data\tifs\test_area_DHMVIIDTMRAS1m_k32_rescaled_2m.tif"
output_csv = r"C:\Users\sarah\OneDrive\Documenten\GitHub\thesis-cds-github\Data\test_area_local_minima.csv"

# Parameters
window_size = 5

# Load and process DEM
try:
    dem_array, geotransform = load_data(dem_path)
    dem_normalized = normalize_data(dem_array)

    # Detect local minima
    start_time = time.time()
    mask = detect_local_minima(dem_normalized, window_size)
    depressions = extract_coordinates(mask, geotransform, window_size)
    elapsed_time = time.time() - start_time

    # Output results
    print(f"Window size: {window_size}")
    print(f"Number of depressions identified: {len(depressions)}")
    print(f"Time taken: {elapsed_time:.2f} seconds")
    save_to_csv(depressions, output_csv, ['id', 'longitude', 'latitude'])
    
except Exception as e:
    print(f"Error: {e}")

Window size: 5
Number of depressions identified: 121475
Time taken: 17.41 seconds
Results saved to C:\Users\sarah\OneDrive\Documenten\GitHub\thesis-cds-github\Data\test_area_local_minima.csv
